In [50]:
import os
import re
import numpy as np
from tqdm import tqdm
import pandas as pd

In [51]:
np.random.seed(42)

In [4]:
MAX_LEN = 20
CLASS_NUMBER = 9
BATCH_SIZE = 10
NUM_EPOCHS = 100
VALID_SPLIT = 0.2
BERT_CKPT = './data_out/'
DATA_IN_PATH = '../metadata/'
DATA_OUT_PATH = './data_out/'

In [53]:
lyrics2 = pd.read_csv(DATA_IN_PATH+'Song_Lyrics2.csv')

In [54]:
lyrics2

,IDX,SONG_ID,LYRICS
0,14,2466,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...
1,163,5112,제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...
2,316,7712,당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...
3,464,9270,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...
4,486,9302,\N
...,...,...,...
3740,348575,35353251,그대 아무런 말도 하지 마요\n이 맘은 여전히 그대로예요\n따가운 햇살 그 아래 우...
3741,348722,35353557,대충 모자를 눌러 쓰고서\n부은 눈을 가리고 나와\n오늘부터는 울어도\n나 때문에 ...
3742,349518,35380072,지금 시각 열한 시 이십팔 분\n물론 아니지 아침\n하루 일과 다 마치구\n우연히 ...
3743,349569,35389041,"Come and, talk to me baby\n뭐든 콕 찝어 말을 해줘\n사소한 ..."


In [55]:
lyrics2.isnull().sum()

IDX        0
SONG_ID    0
LYRICS     5
dtype: int64

In [56]:
lyrics2 = lyrics2.dropna()

In [57]:
def preprocessing(x):
    temp = x.split('\n')
    temp = ' '.join(temp)
    return temp

In [58]:
lyrics2['Edited_LYRICS'] = lyrics2['LYRICS'].apply(preprocessing)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
lyrics2.drop(lyrics2[~lyrics2['Edited_LYRICS'].str.contains(r'([가-힣]+)')].index, inplace=True)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.
c:\project_huiyeol\project\venv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [60]:
def slicing(x):
    x = x.split()
    res = []
    for i in range(0, len(x)+1, MAX_LEN):
        if len(x)-i-MAX_LEN < MAX_LEN//2:
            temp = x[i:]
            res.append(temp)
            break
        temp = x[i:i+MAX_LEN]
        res.append(temp)
    return res

In [61]:
lyrics2['slice'] = lyrics2['Edited_LYRICS'].apply(slicing)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
lyrics2['words'] = lyrics2['Edited_LYRICS'].apply(lambda x:x.split())

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
length = lyrics2['words'].apply(lambda x:len(x))
length.mean()

213.90028046120287

In [64]:
lyrics2['Sliced_LYRICS'] = ''

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
lyrics2

,IDX,SONG_ID,LYRICS,Edited_LYRICS,slice,words,Sliced_LYRICS
0,14,2466,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,나도 알지 못한날 굳이 알려고 들지마 보여줄 수 없는 내 마음만 안타까울 뿐 이...,"[[나도, 알지, 못한날, 굳이, 알려고, 들지마, 보여줄, 수, 없는, 내, 마음...","[나도, 알지, 못한날, 굳이, 알려고, 들지마, 보여줄, 수, 없는, 내, 마음만...",
1,163,5112,제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...,제각기 있기 였기 때문이기 사랑했기 때문이기 하루지기 이야기 이야기 잊지않기 ...,"[[제각기, 있기, 였기, 때문이기, 사랑했기, 때문이기, 하루지기, 이야기, 이야...","[제각기, 있기, 였기, 때문이기, 사랑했기, 때문이기, 하루지기, 이야기, 이야기...",
2,316,7712,당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...,당신은 모르실 거야 얼마나 사랑했는지 세월이 흘러가면은 그때서 뉘우칠 거야 When...,"[[당신은, 모르실, 거야, 얼마나, 사랑했는지, 세월이, 흘러가면은, 그때서, 뉘...","[당신은, 모르실, 거야, 얼마나, 사랑했는지, 세월이, 흘러가면은, 그때서, 뉘우...",
3,464,9270,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...,이렇게 멋진 파란 하늘 위로 날으는 마법 융단을 타고 이렇게 멋진 푸른 세상 속...,"[[이렇게, 멋진, 파란, 하늘, 위로, 날으는, 마법, 융단을, 타고, 이렇게, ...","[이렇게, 멋진, 파란, 하늘, 위로, 날으는, 마법, 융단을, 타고, 이렇게, 멋...",
5,615,10124,It's on and bump it \nIt's on and bump it \nIt...,It's on and bump it It's on and bump it It's...,"[[It's, on, and, bump, it, It's, on, and, bump...","[It's, on, and, bump, it, It's, on, and, bump,...",
...,...,...,...,...,...,...,...
3739,348207,35333345,What's wrong?\n홀릴 듯이 난 너를 자극해 (Pop pop pop) \n...,What's wrong? 홀릴 듯이 난 너를 자극해 (Pop pop pop) Wa...,"[[What's, wrong?, 홀릴, 듯이, 난, 너를, 자극해, (Pop, po...","[What's, wrong?, 홀릴, 듯이, 난, 너를, 자극해, (Pop, pop...",
3740,348575,35353251,그대 아무런 말도 하지 마요\n이 맘은 여전히 그대로예요\n따가운 햇살 그 아래 우...,그대 아무런 말도 하지 마요 이 맘은 여전히 그대로예요 따가운 햇살 그 아래 우리 ...,"[[그대, 아무런, 말도, 하지, 마요, 이, 맘은, 여전히, 그대로예요, 따가운,...","[그대, 아무런, 말도, 하지, 마요, 이, 맘은, 여전히, 그대로예요, 따가운, ...",
3741,348722,35353557,대충 모자를 눌러 쓰고서\n부은 눈을 가리고 나와\n오늘부터는 울어도\n나 때문에 ...,대충 모자를 눌러 쓰고서 부은 눈을 가리고 나와 오늘부터는 울어도 나 때문에 울어 ...,"[[대충, 모자를, 눌러, 쓰고서, 부은, 눈을, 가리고, 나와, 오늘부터는, 울어...","[대충, 모자를, 눌러, 쓰고서, 부은, 눈을, 가리고, 나와, 오늘부터는, 울어도...",
3742,349518,35380072,지금 시각 열한 시 이십팔 분\n물론 아니지 아침\n하루 일과 다 마치구\n우연히 ...,지금 시각 열한 시 이십팔 분 물론 아니지 아침 하루 일과 다 마치구 우연히 접한 ...,"[[지금, 시각, 열한, 시, 이십팔, 분, 물론, 아니지, 아침, 하루, 일과, ...","[지금, 시각, 열한, 시, 이십팔, 분, 물론, 아니지, 아침, 하루, 일과, 다...",


In [66]:
new_data = pd.DataFrame(lyrics2.iloc[0][:]).T
for idx in tqdm(lyrics2.index):
    try:
        for i in range(len(lyrics2.loc[idx]['slice'])):
            temp = lyrics2.loc[idx]['slice'][i]
            txt = ' '.join(temp)
            lyrics2.loc[idx, 'Sliced_LYRICS'] = txt
            new_data = pd.concat([new_data, pd.DataFrame(lyrics2.loc[idx]).T])
    except:
        continue

  0%|          | 0/3209 [00:00<?, ?it/s]c:\project_huiyeol\project\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 3209/3209 [01:37<00:00, 32.97it/s]


In [67]:
new_data.reset_index(drop=True, inplace=True)

In [68]:
new_data = new_data.drop(new_data[new_data['Sliced_LYRICS']==''].index)

In [69]:
new_data = new_data.drop(new_data[new_data['Sliced_LYRICS']=='None'].index)

In [70]:
new_data = new_data.drop(new_data[new_data['Sliced_LYRICS']=='\\N'].index)

In [71]:
lyrics = new_data[['SONG_ID', 'Sliced_LYRICS']]

In [72]:
lyrics['대분류1']=''
lyrics['대분류2']=''

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
# from sklearn.model_selection import train_test_split

# data_train, data_valid = train_test_split(lyrics, test_size=VALID_SPLIT, random_state = 42)

In [24]:
# data_train.sort_index(inplace=True)
# data_valid.sort_index(inplace=True)

In [25]:
# data_train.to_csv(DATA_OUT_PATH+'train_sliced60Lyrics.csv', encoding='utf-8-sig')

In [73]:
lyrics.to_csv(DATA_OUT_PATH+'sliced20Lyrics.csv', index=False, encoding='utf-8-sig')

In [77]:
lyrics.iloc[10000:20000,:-1].to_csv(DATA_OUT_PATH+'sliced20Lyrics10001~20000.csv', index=False, encoding='utf-8-sig')

In [79]:
for i in range(10000,20000,50):
    lyrics.iloc[i:i+50,:-1].to_csv(DATA_OUT_PATH+'sliced20Lyrics'+str(i+1)+'~'+str(i+50)+'.csv', index=False, encoding='utf-8-sig')